In [1]:
import os
import pandas as pd
from strsimpy.levenshtein import Levenshtein
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, Cupid, DistributionBased, JaccardLevenMatcher, SimilarityFlooding


In [39]:
mapping_columns = {
    "name": ["model", "name", "Tên"],
    "url": ["url", "short-link", "link"],
    "cpu": ["chipset", "Bộ xử lý trung tâm", "chip_set", "chip", "chip xử lý", "Chipset (hãng SX CPU)"],
    "os": ["os", "hệ điều hành"],
    "gpu": ["Chip đồ họa", "gpu", "chip_do_hoa"],
    "ram": ["ram", "RAM", "Ram"],
    "rom": ["Bộ nhớ trong", "rom"],
    "price": ["giá", "price"],
    "connect": ["kết nối", "connect"],
    "screentype": ["công nghệ màn hình", "loại màn hình", "display_type", "chất lượng màn hình", "kiểu màn hình"],
    "color": ["color", "color-split", "màu"],
    "front_camera": ["camera phụ", "camera_truoc"],
    "rear_camera": ["camera chính", "camera_sau"],
    "battery_capacity": ["dung lượng pin", "pin"],
    "bluetooth": ["bluetooth"],
    "wifi": ["wifi", "Wifi"],
    "size": ["màn hình rộng", "kích thước màn hình"],
    "resolution": ["resolution", "độ phân giải", "độ phân giải màn hình"],
    "weight": ["trọng lượng", "product_weight", "cân nặng", "Kích thước khối lượng"],
    "jack_audio": ["jack tai nghe"],
    "mobile_network": ["mạng di động", "hỗ trợ 5G"],
    "sim": ["sim"],
    "security": ["bảo mật nâng cao", "security"],
    "material": ["material", "chất liệu"]
}
target_schema = []
for col in mapping_columns:
    target_schema.append(col)

In [3]:
path = "../PreprocessingData/Dataprocess/iphone/ddtm.json"
df = pd.read_json(path)

In [9]:
list_columns = list(df.columns)
columns = list_columns[0:43]

In [13]:

df2 = df[columns].drop([" Màn hình rộng "], axis=1)

In [ ]:
columns = list(df2.columns)
columns

In [3]:
levenshtein = Levenshtein()

def similarity_score(x, y):
    x = x.lower().strip()
    y = y.lower().strip()
    return (len(x) + len(y) - levenshtein.distance(x, y)) / (len(x) + len(y))
def check_similarity(x, y, threshold=0.8):
    score = similarity_score(x, y)
    if score > threshold:
        return True
    return False 

In [17]:
threshold = 0.85

res = dict()
for colum in columns:
    for key in mapping_columns:
        listMatching = mapping_columns[key]
        for match in listMatching:
            score = similarity_score(colum, match)
            if(score > threshold):
                print("{0:20}\t{0:20}\t".format(colum, match) + str(score))
                res[colum] = key
        

Bluetooth           	Bluetooth           	1.0
Bảo mật nâng cao    	Bảo mật nâng cao    	1.0
Bộ nhớ trong        	Bộ nhớ trong        	1.0
Chip xử lý          	Chip xử lý          	1.0
Chip đồ họa         	Chip đồ họa         	1.0
Công nghệ màn hình  	Công nghệ màn hình  	1.0
Dung lượng pin      	Dung lượng pin      	1.0
Jack tai nghe       	Jack tai nghe       	1.0
Kích thước khối lượng 	Kích thước khối lượng 	1.0
Model-split         	Model-split         	1.0
Màn hình rộng       	Màn hình rộng       	1.0
Mạng di động        	Mạng di động        	1.0
RAM                 	RAM                 	1.0
RAM                 	RAM                 	1.0
RAM                 	RAM                 	1.0
Rom-split           	Rom-split           	1.0
Sim                 	Sim                 	1.0
Wifi                	Wifi                	1.0
Wifi                	Wifi                	1.0
color               	color               	1.0
name                	name                	1.0
price               	price    

In [18]:
res

{'Bluetooth ': 'bluetooth',
 'Bảo mật nâng cao ': 'security',
 'Bộ nhớ trong ': 'rom',
 'Chip xử lý ': 'cpu',
 'Chip đồ họa ': 'gpu',
 'Công nghệ màn hình ': 'screentype',
 'Dung lượng pin ': 'battery_capacity',
 'Jack tai nghe ': 'jack_audio',
 'Kích thước khối lượng ': 'weight',
 'Model-split': 'name',
 'Màn hình rộng ': 'size',
 'Mạng di động ': 'mobile_network',
 'RAM ': 'ram',
 'Rom-split': 'rom',
 'Sim ': 'sim',
 'Wifi ': 'wifi',
 'color': 'color',
 'name': 'name',
 'price': 'price',
 'url': 'url',
 'Độ phân giải ': 'resolution'}

In [19]:

data = pd.DataFrame(columns=target_schema)

In [20]:
tmp = res.pop("name")
tmp2 = res.pop("Bộ nhớ trong ")
res


{'Bluetooth ': 'bluetooth',
 'Bảo mật nâng cao ': 'security',
 'Chip xử lý ': 'cpu',
 'Chip đồ họa ': 'gpu',
 'Công nghệ màn hình ': 'screentype',
 'Dung lượng pin ': 'battery_capacity',
 'Jack tai nghe ': 'jack_audio',
 'Kích thước khối lượng ': 'weight',
 'Model-split': 'name',
 'Màn hình rộng ': 'size',
 'Mạng di động ': 'mobile_network',
 'RAM ': 'ram',
 'Rom-split': 'rom',
 'Sim ': 'sim',
 'Wifi ': 'wifi',
 'color': 'color',
 'price': 'price',
 'url': 'url',
 'Độ phân giải ': 'resolution'}

In [23]:
matched_col = []
source_col = []
for key in res:
    matched_col.append(res[key])
    source_col.append(key)

In [24]:
source_col

['Bluetooth ',
 'Bảo mật nâng cao ',
 'Chip xử lý ',
 'Chip đồ họa ',
 'Công nghệ màn hình ',
 'Dung lượng pin ',
 'Jack tai nghe ',
 'Kích thước khối lượng ',
 'Model-split',
 'Màn hình rộng ',
 'Mạng di động ',
 'RAM ',
 'Rom-split',
 'Sim ',
 'Wifi ',
 'color',
 'price',
 'url',
 'Độ phân giải ']

In [25]:
df2_tmp = df2[source_col]
df2_tmp = df2_tmp.rename(columns=res)
df2_tmp.head(5)

,bluetooth,security,cpu,gpu,screentype,battery_capacity,jack_audio,weight,name,size,mobile_network,ram,rom,sim,wifi,color,price,url,resolution
0,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Xanh lá,11.640.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels
1,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Đen,11.640.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels
2,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Tím,11.640.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels
3,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Đỏ,11.890.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels
4,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Trắng,11.390.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels


In [26]:
df2_tmp.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584 entries, 0 to 1583
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   bluetooth         1418 non-null   object
 1   security          1418 non-null   object
 2   cpu               1283 non-null   object
 3   gpu               1400 non-null   object
 4   screentype        1418 non-null   object
 5   battery_capacity  1418 non-null   object
 6   jack_audio        1402 non-null   object
 7   weight            1418 non-null   object
 8   name              1584 non-null   object
 9   size              1418 non-null   object
 10  mobile_network    1402 non-null   object
 11  ram               1328 non-null   object
 12  rom               1572 non-null   object
 13  sim               1418 non-null   object
 14  wifi              1418 non-null   object
 15  color             1584 non-null   object
 16  price             1584 non-null   object
 17  url           

In [27]:
data = pd.DataFrame(columns=target_schema)
data

,name,url,cpu,os,gpu,ram,rom,price,connect,screentype,...,bluetooth,wifi,size,resolution,weight,jack_audio,mobile_network,sim,security,material


In [29]:
data[matched_col] = df2_tmp

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584 entries, 0 to 1583
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              1584 non-null   object
 1   url               1584 non-null   object
 2   cpu               1283 non-null   object
 3   os                0 non-null      object
 4   gpu               1400 non-null   object
 5   ram               1328 non-null   object
 6   rom               1572 non-null   object
 7   price             1584 non-null   object
 8   connect           0 non-null      object
 9   screentype        1418 non-null   object
 10  color             1584 non-null   object
 11  front_camera      0 non-null      object
 12  rear_camera       0 non-null      object
 13  battery_capacity  1418 non-null   object
 14  bluetooth         1418 non-null   object
 15  wifi              1418 non-null   object
 16  size              1418 non-null   object
 17  resolution    

In [32]:
data.to_json("./IntegratedData/target_data.json")

In [33]:
data 

,name,url,cpu,os,gpu,ram,rom,price,connect,screentype,...,bluetooth,wifi,size,resolution,weight,jack_audio,mobile_network,sim,security,material
0,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.640.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
1,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.640.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
2,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.640.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
3,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.890.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
4,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.390.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,IPHONE 11,https://didongthongminh.vn/iphone-11-128gb-chi...,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,128GB,13.190.000đ,NaN,IPS LCD,...,A2DPv5.0,Wi-Fi hotspotWi-Fi 802.11 a/b/g/n/ac/axDual-ba...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
1580,IPHONE 11,https://didongthongminh.vn/iphone-11-128gb-chi...,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,128GB,13.190.000đ,NaN,IPS LCD,...,A2DPv5.0,Wi-Fi hotspotWi-Fi 802.11 a/b/g/n/ac/axDual-ba...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
1581,IPHONE 11,https://didongthongminh.vn/iphone-11-128gb-chi...,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,128GB,13.290.000đ,NaN,IPS LCD,...,A2DPv5.0,Wi-Fi hotspotWi-Fi 802.11 a/b/g/n/ac/axDual-ba...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
1582,IPHONE 11,https://didongthongminh.vn/iphone-11-128gb-chi...,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,128GB,13.190.000đ,NaN,IPS LCD,...,A2DPv5.0,Wi-Fi hotspotWi-Fi 802.11 a/b/g/n/ac/axDual-ba...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN


In [11]:
def schema_matching(cols, map_cols, threshold=0.85):

    res = dict()
    for colum in cols:
        for key in map_cols:
            listMatching = map_cols[key]
            for match in listMatching:
                score = similarity_score(colum, match)
                if(score > threshold):
                    print("{0:20}\t{0:20}\t".format(colum, match) + str(score))
                    res[colum] = key
    return res
            

# xtmobile.json

In [40]:
path = "../PreprocessingData/Dataprocess/iphone/xtmobile.json"
df = pd.read_json(path)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   name                   329 non-null    object        
 1   url                    329 non-null    object        
 2   Công nghệ màn hình     329 non-null    object        
 3   Độ phân giải           329 non-null    object        
 4   Màn hình rộng          329 non-null    object        
 5   Mặt kính cảm ứng       329 non-null    object        
 6   Kiểu màn hình          329 non-null    object        
 7   Quay phim              329 non-null    object        
 8   Đèn Flash              329 non-null    object        
 9   Chụp ảnh nâng cao      329 non-null    object        
 10  Videocall              329 non-null    object        
 11  Tính năng khác         329 non-null    object        
 12  Hệ điều hành           329 non-null    object        
 13  Chips

In [44]:
df2 = df.drop(["Kiểu màn hình"], axis=1)
columns = list(df2.columns)

In [45]:
res = schema_matching(columns, mapping_columns, threshold=0.85)
res

name                	name                	1.0
url                 	url                 	1.0
Công nghệ màn hình  	Công nghệ màn hình  	1.0
Độ phân giải        	Độ phân giải        	1.0
Màn hình rộng       	Màn hình rộng       	1.0
Hệ điều hành        	Hệ điều hành        	1.0
Chipset (hãng SX CPU)	Chipset (hãng SX CPU)	1.0
RAM                 	RAM                 	1.0
RAM                 	RAM                 	1.0
RAM                 	RAM                 	1.0
Bộ nhớ trong        	Bộ nhớ trong        	1.0
Mạng di động        	Mạng di động        	1.0
SIM                 	SIM                 	1.0
Wifi                	Wifi                	1.0
Wifi                	Wifi                	1.0
Bluetooth           	Bluetooth           	1.0
Jack tai nghe       	Jack tai nghe       	1.0
Chất liệu           	Chất liệu           	1.0
Trọng lượng         	Trọng lượng         	1.0
Dung lượng pin      	Dung lượng pin      	1.0
Bảo mật nâng cao    	Bảo mật nâng cao    	1.0
color               	color      

{'name': 'name',
 'url': 'url',
 'Công nghệ màn hình': 'screentype',
 'Độ phân giải': 'resolution',
 'Màn hình rộng': 'size',
 'Hệ điều hành': 'os',
 'Chipset (hãng SX CPU)': 'cpu',
 'RAM': 'ram',
 'Bộ nhớ trong': 'rom',
 'Mạng di động': 'mobile_network',
 'SIM': 'sim',
 'Wifi': 'wifi',
 'Bluetooth': 'bluetooth',
 'Jack tai nghe': 'jack_audio',
 'Chất liệu': 'material',
 'Trọng lượng': 'weight',
 'Dung lượng pin': 'battery_capacity',
 'Bảo mật nâng cao': 'security',
 'color': 'color',
 'price': 'price'}

In [46]:
def matchedCol(result): 
    matched_col = []
    source_col = []
    for key in result:
        matched_col.append(result[key])
        source_col.append(key)
    
    return matched_col, source_col

In [47]:
matched_col, source_col = matchedCol(res)

In [51]:
df_tmp = df2[source_col].rename(columns=res)
df_tmp

,name,url,screentype,resolution,size,os,cpu,ram,rom,mobile_network,sim,wifi,bluetooth,jack_audio,material,weight,battery_capacity,security,color,price
0,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Super Retina XDR,12MP True Depth,6.1 inch,iOS 15,Apple A14 Bionic,4GB,64GB,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Wi‑Fi 6 (802.11ax),Bluetooth 5.0,Lightning,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính,164g,2.815mAh,Nhận diện khuôn mặt Face ID,Đen,14.190.000đ
1,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Super Retina XDR,12MP True Depth,6.1 inch,iOS 15,Apple A14 Bionic,4GB,64GB,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Wi‑Fi 6 (802.11ax),Bluetooth 5.0,Lightning,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính,164g,2.815mAh,Nhận diện khuôn mặt Face ID,Trắng,14.390.000đ
2,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Super Retina XDR,12MP True Depth,6.1 inch,iOS 15,Apple A14 Bionic,4GB,64GB,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Wi‑Fi 6 (802.11ax),Bluetooth 5.0,Lightning,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính,164g,2.815mAh,Nhận diện khuôn mặt Face ID,Tím,14.590.000đ
3,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Super Retina XDR,12MP True Depth,6.1 inch,iOS 15,Apple A14 Bionic,4GB,64GB,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Wi‑Fi 6 (802.11ax),Bluetooth 5.0,Lightning,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính,164g,2.815mAh,Nhận diện khuôn mặt Face ID,Đỏ,14.590.000đ
4,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Super Retina XDR,12MP True Depth,6.1 inch,iOS 15,Apple A14 Bionic,4GB,64GB,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Wi‑Fi 6 (802.11ax),Bluetooth 5.0,Lightning,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính,164g,2.815mAh,Nhận diện khuôn mặt Face ID,Xanh Lá,14.590.000đ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,iPhone SE 2022 128GB Chính hãng (VN/A),https://www.xtmobile.vn/iphone-se-2022-5g-128gb,Retina IPS LCD,"7 MP, f/2.2",4.7 inches,iOS 15.3,Apple A15 Bionic (5 nm) Hexa-core,3 GB,128 GB,Hỗ trợ 5G,Nano-SIM and/or eSIM,"Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot","5.0, A2DP, LE",,,,,,Đen,13.190.000đ
325,iPhone SE 2022 256GB Chính hãng (VN/A),https://www.xtmobile.vn/iphone-se-2022-5g-256gb,Retina IPS LCD,"7 MP, f/2.2",4.7 inches,iOS 15.3,Apple A15 Bionic (5 nm) Hexa-core,3 GB,64 GB,Hỗ trợ 5G,Nano-SIM and/or eSIM,"Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot","5.0, A2DP, LE",,,,,,Đỏ,15.990.000đ
326,iPhone SE 2022 256GB Chính hãng (VN/A),https://www.xtmobile.vn/iphone-se-2022-5g-256gb,Retina IPS LCD,"7 MP, f/2.2",4.7 inches,iOS 15.3,Apple A15 Bionic (5 nm) Hexa-core,3 GB,64 GB,Hỗ trợ 5G,Nano-SIM and/or eSIM,"Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot","5.0, A2DP, LE",,,,,,Trắng,15.990.000đ
327,iPhone SE 2022 256GB Chính hãng (VN/A),https://www.xtmobile.vn/iphone-se-2022-5g-256gb,Retina IPS LCD,"7 MP, f/2.2",4.7 inches,iOS 15.3,Apple A15 Bionic (5 nm) Hexa-core,3 GB,64 GB,Hỗ trợ 5G,Nano-SIM and/or eSIM,"Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot","5.0, A2DP, LE",,,,,,Đen,15.990.000đ


In [52]:
data = pd.DataFrame(columns=target_schema)
data

,name,url,cpu,os,gpu,ram,rom,price,connect,screentype,...,bluetooth,wifi,size,resolution,weight,jack_audio,mobile_network,sim,security,material


In [54]:
data[matched_col] = df_tmp[matched_col]

In [55]:
data

,name,url,cpu,os,gpu,ram,rom,price,connect,screentype,...,bluetooth,wifi,size,resolution,weight,jack_audio,mobile_network,sim,security,material
0,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Apple A14 Bionic,iOS 15,NaN,4GB,64GB,14.190.000đ,NaN,Super Retina XDR,...,Bluetooth 5.0,Wi‑Fi 6 (802.11ax),6.1 inch,12MP True Depth,164g,Lightning,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Nhận diện khuôn mặt Face ID,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính
1,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Apple A14 Bionic,iOS 15,NaN,4GB,64GB,14.390.000đ,NaN,Super Retina XDR,...,Bluetooth 5.0,Wi‑Fi 6 (802.11ax),6.1 inch,12MP True Depth,164g,Lightning,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Nhận diện khuôn mặt Face ID,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính
2,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Apple A14 Bionic,iOS 15,NaN,4GB,64GB,14.590.000đ,NaN,Super Retina XDR,...,Bluetooth 5.0,Wi‑Fi 6 (802.11ax),6.1 inch,12MP True Depth,164g,Lightning,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Nhận diện khuôn mặt Face ID,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính
3,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Apple A14 Bionic,iOS 15,NaN,4GB,64GB,14.590.000đ,NaN,Super Retina XDR,...,Bluetooth 5.0,Wi‑Fi 6 (802.11ax),6.1 inch,12MP True Depth,164g,Lightning,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Nhận diện khuôn mặt Face ID,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính
4,"iPhone 12 64GB (LikeNew 99,99% - FullBox) - Pi...",https://www.xtmobile.vn/iphone-12-64gb-cu,Apple A14 Bionic,iOS 15,NaN,4GB,64GB,14.590.000đ,NaN,Super Retina XDR,...,Bluetooth 5.0,Wi‑Fi 6 (802.11ax),6.1 inch,12MP True Depth,164g,Lightning,"Hỗ trợ LTE, 5G","2 sim (1 sim vật lý, 1 esim)",Nhận diện khuôn mặt Face ID,Khung nhôm + Màn hình mặt gốm + Mặt lưng kính
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,iPhone SE 2022 128GB Chính hãng (VN/A),https://www.xtmobile.vn/iphone-se-2022-5g-128gb,Apple A15 Bionic (5 nm) Hexa-core,iOS 15.3,NaN,3 GB,128 GB,13.190.000đ,NaN,Retina IPS LCD,...,"5.0, A2DP, LE","Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot",4.7 inches,"7 MP, f/2.2",,,Hỗ trợ 5G,Nano-SIM and/or eSIM,,
325,iPhone SE 2022 256GB Chính hãng (VN/A),https://www.xtmobile.vn/iphone-se-2022-5g-256gb,Apple A15 Bionic (5 nm) Hexa-core,iOS 15.3,NaN,3 GB,64 GB,15.990.000đ,NaN,Retina IPS LCD,...,"5.0, A2DP, LE","Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot",4.7 inches,"7 MP, f/2.2",,,Hỗ trợ 5G,Nano-SIM and/or eSIM,,
326,iPhone SE 2022 256GB Chính hãng (VN/A),https://www.xtmobile.vn/iphone-se-2022-5g-256gb,Apple A15 Bionic (5 nm) Hexa-core,iOS 15.3,NaN,3 GB,64 GB,15.990.000đ,NaN,Retina IPS LCD,...,"5.0, A2DP, LE","Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot",4.7 inches,"7 MP, f/2.2",,,Hỗ trợ 5G,Nano-SIM and/or eSIM,,
327,iPhone SE 2022 256GB Chính hãng (VN/A),https://www.xtmobile.vn/iphone-se-2022-5g-256gb,Apple A15 Bionic (5 nm) Hexa-core,iOS 15.3,NaN,3 GB,64 GB,15.990.000đ,NaN,Retina IPS LCD,...,"5.0, A2DP, LE","Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot",4.7 inches,"7 MP, f/2.2",,,Hỗ trợ 5G,Nano-SIM and/or eSIM,,


In [56]:
data.to_json("./IntegratedData/data2.json")

# ViettelStore.json

In [71]:
from SchemaMatchingSystem import SchemaMatchingSystem

In [72]:
schemaMatching = SchemaMatchingSystem()

In [73]:
path = "../PreprocessingData/Dataprocess/iphone/ViettelStore.json"
df = pd.read_json(path)

In [75]:
columns = list(df.columns)
res = schema_matching(cols=columns, map_cols=mapping_columns)
res

name                	name                	1.0
url                 	url                 	1.0
price               	price               	1.0
Chất liệu           	Chất liệu           	1.0
Hỗ trợ 5G           	Hỗ trợ 5G           	1.0
Kích thước màn hình	Kích thước màn hình	0.9
Chipset             	Chipset             	1.0
Chipset             	Chipset             	0.9333333333333333
RAM                 	RAM                 	1.0
RAM                 	RAM                 	1.0
RAM                 	RAM                 	1.0
Camera chính        	Camera chính        	1.0
Camera phụ          	Camera phụ          	1.0
Bộ nhớ trong        	Bộ nhớ trong        	1.0
Wifi                	Wifi                	1.0
Wifi                	Wifi                	1.0
Bluetooth           	Bluetooth           	1.0
Dung lượng Pin     	Dung lượng Pin     	0.9310344827586207


{'name': 'name',
 'url': 'url',
 'price': 'price',
 'Chất liệu': 'material',
 'Hỗ trợ 5G': 'mobile_network',
 'Kích thước màn hình': 'size',
 'Chipset': 'cpu',
 'RAM': 'ram',
 'Camera chính': 'rear_camera',
 'Camera phụ': 'front_camera',
 'Bộ nhớ trong': 'rom',
 'Wifi': 'wifi',
 'Bluetooth': 'bluetooth',
 'Dung lượng Pin': 'battery_capacity'}

In [77]:
matched_col, source_col = matchedCol(res)

In [79]:
df_tmp = df[source_col].rename(columns=res)
data = pd.DataFrame(columns=target_schema)

In [80]:
data[matched_col] = df_tmp[matched_col]

In [82]:
data.to_json("./IntegratedData/data3.json")